In [5]:
import pandas as pd

all_tweets_df = pd.read_csv('all_tweets')
all_tweets_df = all_tweets_df.dropna(axis=0)
all_tweets_df['condition'] = all_tweets_df['condition'].replace({'depression':0, 'ptsd':1, 'control':2})
all_tweets_df = all_tweets_df[pd.to_numeric(all_tweets_df['condition'], errors='coerce').notnull()]

In [56]:
depression_tweets = all_tweets_df.loc[all_tweets_df['condition']!=1]

In [57]:
control_tweets = depression_tweets.loc[depression_tweets['condition']==2][:1132259]

In [58]:
depression_tweets = pd.concat([depression_tweets.loc[depression_tweets['condition']==0], control_tweets])

In [59]:
depression_tweets_copy = depression_tweets

In [61]:
import difflib
from tqdm import tqdm
# Threshold filter based on Percentage similarity

thr = 0.85
depression_tweets['Flag'] = 0
users = depression_tweets['username'].unique()
for name in tqdm(users):
    df = depression_tweets.loc[depression_tweets['username']==name]
    for text in df['text'].tolist():
        df['temp'] = [difflib.SequenceMatcher(None, text1,text).ratio() for text1 in df['text'].tolist()]
        df.loc[df['temp'].gt(thr),['Flag']] = depression_tweets['Flag'].max()+1
    df.drop('temp',1)

depression_tweets.loc[~depression_tweets['Flag'].duplicated(keep='first')]

  0%|          | 0/1003 [00:00<?, ?it/s]/var/folders/g4/q2c7q3553szgh172m493hl4r0000gn/T/ipykernel_33400/123720474.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['temp'] = [difflib.SequenceMatcher(None, text1,text).ratio() for text1 in df['text'].tolist()]
/var/folders/g4/q2c7q3553szgh172m493hl4r0000gn/T/ipykernel_33400/123720474.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['temp'] = [difflib.SequenceMatcher(None, text1,text).ratio() for text1 in df['text'].tolist()]
/var/folders/g4/q2c7

KeyboardInterrupt: 

In [62]:
depression_tweets_copy.to_csv('depression_tweets')

In [81]:
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np

X= np.array(depression_tweets_copy['text'])
y = np.array(depression_tweets_copy['condition'])
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.05, random_state=123)

for train_index, test_index in sss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [85]:
reduced_set = pd.DataFrame({'text': X_test,
                            'label': y_test})

In [87]:
reduced_set.to_csv('reduced_set')